# 协程
 - 参考资料
     - python.jobbole.com/86481/
     - python.jobbole.com/87310/
     - segmentfault.com/a/1190000009781688
## 迭代器
 - 可迭代(Iterable): 直接作用于for循环的变量
 - 迭代器(Iterator): 不但可以作用于for 循环, 还可以被next() 调用
 - list 是典型的可迭代对象, 但是不是迭代器
 
 

In [1]:
# 可迭代
l = [i for i in range(10)]
for idx in l:
    print(idx)
# range shi 迭代器
for i in range(5):
    print(i)


0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


In [5]:
# isinstance 案例
# 判断某个变量是否是一个实例
# 判断是否可迭代
from collections import Iterable, Iterator
l1 = [1, 2, 3, 4]
print(isinstance(l1, Iterable))
print(isinstance(l1, Iterator))

True
False


In [7]:
# iter 函数
s = "好好学习天天向上"
print(isinstance(s, Iterable))
print(isinstance(s, Iterator))
s = iter(s)
print(isinstance(s, Iterator))

True
False
True


# 生成器
 - generator : 一边循环一边计算下一个元素的机制/算法
 - 需要满足三个条件
     - 每次调用都产生出for循环需要的下一个元素
     - 如果达到最后一个后, 爆出StopIteration 异常
     - 可以被next() 函数调用
 - 如何生成一个生成器
     - 直接使用
     - 如果函数中包含yield, 则这个函数就叫生成器
     - next() 调用函数, 遇到yield 返回
 - range() 比较常用且典型的生成器
 

In [17]:
# 生成器  案例 直接使用
L = [x*x for x in range(5)] # 列表生成器
g = (x*x for x in range(5)) # 生成器
print(type(L))
print(type(g))

<class 'list'>
<class 'generator'>


In [20]:
# 函数生成器案例
def func():
    n = 0
    while n < 100:
        yield n
        n +=1


In [25]:
f = func()
for i in range(5):
    print(next(f))
for i in range(5):
    print(next(f))


0
1
2
3
4
5
6
7
8
9


In [30]:
# Fibonacci 数求接
def fib():
    a, b =  0, 1
    while True:
        yield b
        a, b = b, a+b
        
g = fib()
for i in range(10):
    print(next(g))
        

1
1
2
3
5
8
13
21
34
55


# 协程
 - 历史
     - 3.4 引入协程, 用yield 实现
     - 3.5 引入协程语法
     - 实现多额协程比较好的包有, asynio, tornado, gevent
 - 定义:
     - 是为了非抢占式多任务产生子程序的计算机程序组件, 协程允许不同入口点在不同位置暂停或者开始执行任务
     
 - 从技术角度讲, 协程就是一个你可以暂停执行的函数, 或者干脆把协程理解为生成器
 - 协程的实现:
     - yield 返回
     - send 调用
 - 协程的四个状态
     - insppect.getgeneratorstate(...) 函数确定, 该函数会返回下雨字符串中的一个
     - GEN_CREATED: 等待开始执行
     - GEN_RUNNING: 解释器正在执行
     - GEN_SUPPENED: 在yield 表达式处暂停
     - GEN-CLOSED: 执行结束
     - next 预激 (prime)
 - 协程终止
     - 协程中未处理的异常会向上冒泡, 传给next 函数 或send 方法的调用(即储大协程的对象)
     - 终止协程的一种方式: 发送莫格哨符值, 让协程退出. 内置的None 和Ellipsis 等常量经常用作哨符值==.
 - yield from
     - 调用协程为了得到返回值, 协程必须正常终止
     - 生成器正常终止, 会发出StopItertion 异常, 异常对象的vlaue 属性保存返回值
     - yield from从内部捕获StopIteration 异常
     - 委派生成器
         - 包含yield from 表达式的生成器函数
         - 委派生成器 在yield from 表达式出暂停, 调用方可以直接把数据发给子生成器
         - 子生成器,在把产出的值发给调用方
         - 子生成器在最后, 解释器会抛出StopIteration, 并返回值附加到对象上去
  

In [33]:
# 协程代码实现
def simple_coroutine():
    print("启动")
    x = yield
    print("结束", x)

print("主线程开启")
sc = simple_coroutine() # 创建协程
next(sc) # 预激
print("结束!")
sc.send("我走了")


主线程开启
启动
结束!
结束 我走了


StopIteration: 

In [1]:
# 协程案例2
def func(a):
    print("开始了")
    b = yield a
    print("-->", a, b)
    c = yield a+b
    print("-->", a, b, c)
    d = yield a+b+c
    
sc = func(5)
aa = next(sc)
print(aa)
bb = sc.send(6)
print(bb)
cc = sc.send(7)
print(cc)


开始了
5
--> 5 6
11
--> 5 6 7
18


In [3]:
# yield from 案例
def g():
    for i in "AB":
        yield i
print(list(g()))

def f():
    yield from "AB"
print(list(f()))

['A', 'B']
['A', 'B']


In [7]:
# 委派生成器
"""
解释: 
1. 外层for 循环每次迭代,会新建一个grouper 实例, 赋值给coroutine 变量, grouper是委派生成器
2. 调用next(coroutine), 预激 委派生成器grouper,此时进入到while循环,调用子生成器averager()
3. 内层for 循环调用coroutine.send(value), 直接把值传给 子生成器averager. 同时当前grouper
4. 内层循环结束后, grouper 实例在yield from 表达式处暂停, 因此, grouper 函数定义体中
5. corutine.send(None) 终止 averager 子生成器, 子生成器抛出StopIteration 异常并放回的数值

"""
from collections import namedtuple
ResClass = namedtuple("Res", "count average")

# 子生成器
def averger():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return ResClass(count, average)

# 委派生成器
def grouper(storages, key):
    while True:
        storages[key] = yield from averger()

# 客户端代码
def client():
    pricess_date = {
        "boys_1" : [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        "boys_2" : [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]   
    }
    
    storages = {}
    for k, v in pricess_date.items():
        # 创建一个协程
        coroutine = grouper(storages, k)
        # 预激
        next(coroutine)
        for dt in v:
            # 发送数据到协程
            coroutine.send(dt)
        # 结束协程
        coroutine.send(None)
    print(storages)

client()
        

{'boys_1': Res(count=9, average=40.422222222222224), 'boys_2': Res(count=9, average=1.3888888888888888)}


## asyncio
 - python 3.4 开始引入标准库当中, 内置对异步io的支持
 - asyncio 本身式一个消息循环
 - 步骤:
     - 创建消息循环
     - 把协程导入
     - 关闭

## async and await
- 为了更好的表示异步io
- python3.5 引入
- 让协程代码更简单
- 使用上, 可以简单的进行替换
    - 用 async 替换@asyncio.coroutine
    - await 替换yield from
    


## aiohttp
 - asyncio 实现单线程的并发io, 在客户端用处不大
 - 在服务器端可以 asyncio + coroutine 配合, 因为http 式io操作
 - asyncio 实现tcp, udp, ssl 等协议
 - aiohttp 是给与asyncio 实现http 框架
 - pip install ainhttp 安装

## concurrent.futures
 - python 3 新增的库`
 - 类似其他语言的线程池 概念
 - 利用multiprocessiong 实现真正的并行计算
 - 核心原理: 以子进程的形式,并行运行多个解释器从而令子进程与主进程直接按分离, 所以他们的全局解释器锁也是相互独立的, 每个子进程都能够完全的使用cpu 内核
 - concurrent.futures.Executor
     - ThreadPoolExecutor
     - ProcessPoolExecutor
     - 执行的时候需要自行选择
     
 - submit(fn, args, kwargs)
     - fn: 异步执行的函数
     - args, kwargs : 参数
                             

In [15]:
# concurrent.futures 案例
from concurrent.futures import ThreadPoolExecutor
import time
def run_future(msg):
    print("上啊")
    time.sleep(4)
    return msg
# 创建线程池
pool = ThreadPoolExecutor(max_workers=2)
# 往线程池加入2个task
f1 = pool.submit(run_future, "你好")
f2 = pool.submit(run_future, "世界")

# 线程是否完成
print(f1.done())
time.sleep(3)
print(f2.done())

print(f1.result())
print(f2.result())

上啊
上啊False

False
你好
世界


## current 中的map 函数
- map(fn, \*iterabls, timeout=None)
    - 和map函数类似
    
    - 函数需要异步执行
    - timeout: 超时时间
    - map 和submit 使用一个就行
## Future


In [ ]:
# 案例
